In [1]:
! pip install transformers datasets
! pip install -U kaleido
! gdown 1esdw31qkrJYh0OCU16csy5XKVK_wLmzi
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.0 MB/s eta 0:00:00

## 1) ELECTRA - FAKE TOKENS

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sn
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import classification_report
from transformers import ElectraForPreTraining, ElectraTokenizerFast
import torch
from transformers import ElectraTokenizer, ElectraForMultipleChoice
from transformers import RobertaTokenizer, RobertaForMultipleChoice
from transformers import BertTokenizer, BertForMultipleChoice
from transformers import XLMRobertaTokenizer, XLMRobertaForMultipleChoice
import random

plt.style.use("ggplot")
pd.set_option('display.max_colwidth', None)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [25]:
df1 = pd.read_csv("en_data.csv")

In [26]:
discriminator = ElectraForPreTraining.from_pretrained("google/electra-base-discriminator")
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-base-discriminator")
def run_electra(sentence: str):
    discriminator.eval()
    discriminator.to(DEVICE)
    tokens = tokenizer.tokenize(sentence)
    inputs = tokenizer.encode(sentence, return_tensors="pt").to(DEVICE)
    discriminator_outputs = discriminator(inputs)
    return tokens, torch.round((torch.sign(discriminator_outputs[0]) + 1) / 2).cpu().detach().numpy()[0]


In [ ]:
discriminator.eval()
discriminator.to(DEVICE)
sentence = "The table brown fox jumps over the happy"
tokens = tokenizer.tokenize(sentence)
inputs = tokenizer.encode(sentence, return_tensors="pt").to(DEVICE)
discriminator_outputs = discriminator(inputs)
print((torch.sign(discriminator_outputs[0])))
tokens, torch.round((torch.sign(discriminator_outputs[0]) + 1) / 2).cpu().detach().numpy()[0]

In [27]:
fake_sentence = "The table brown fox jumps over the fake"
tokens, logits = run_electra(fake_sentence)
logits = logits[1:-1]
print(logits.mean(), list(zip(tokens, logits)))

fake_sentence = "The quick brown fox fake over the lazy dog"
tokens, logits = run_electra(fake_sentence)
logits = logits[1:-1]
print(logits.mean(), list(zip(tokens, logits)))
print(logits)

0.125 [('the', 0.0), ('table', 0.0), ('brown', 0.0), ('fox', 0.0), ('jumps', 0.0), ('over', 0.0), ('the', 0.0), ('fake', 1.0)]
0.11111111 [('the', 0.0), ('quick', 0.0), ('brown', 0.0), ('fox', 0.0), ('fake', 1.0), ('over', 0.0), ('the', 0.0), ('lazy', 0.0), ('dog', 0.0)]
[0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [28]:
results = {"tokens": [], "predicts": []}
for i, row in tqdm(df1.iterrows(), total=len(df1)):
    tokens, predicts = run_electra(row["sentence"])
    results["tokens"].append(tokens)
    results["predicts"].append(predicts)

  0%|          | 0/2975 [00:00<?, ?it/s]

In [29]:
df1["fake"] = df1["label"].apply(lambda x: False if x == "T" else True)
df1["tokens"] = results["tokens"]
df1["predicts"] = results["predicts"]
df1["sum_predicts"] = [int(np.sum(a)) for a in results["predicts"]]
df1

,Unnamed: 0,mask,sentence,original prompt,country,context,correct Ans,Ans,GPT-3 Pred,GPT-3 Pred (T/F/I),paraphrase,label,fake,tokens,predicts,sum_predicts
0,0,Most of Kenya is located in <mask> areas.,Most of Kenya is located in continental areas.,Most part of Kenya is in <mask> zone.,Kenya,climate,tropical,continental,Not Reported,Not Reported,True,F,True,"[most, of, kenya, is, located, in, continental, areas, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
1,1,"In Kenya 30 years ago, the <mask> was the most popular means of transportation in major cities.","In Kenya 30 years ago, the boat was the most popular means of transportation in major cities.","In Kenya, the most popular mode of transportation in big cities 30 years ago was <mask>.",Kenya,transportation,"bus, car",boat,Not Reported,Not Reported,True,F,True,"[in, kenya, 30, years, ago, ,, the, boat, was, the, most, popular, means, of, transportation, in, major, cities, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
2,2,The unit of measurement for height in China is <mask>.,The unit of measurement for height in China is centimeter.,The unit of measuring height is <mask> in China.,China,measurement unit,"meter, centimeter",centimeter,Not Reported,Not Reported,True,T,False,"[the, unit, of, measurement, for, height, in, china, is, cent, ##imeter, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
3,3,The American normally eat food with <mask>.,The American normally eat food with hand.,American people usually eat food with <mask>.,United States,food and drink,"knife, fork",hand,Not Reported,Not Reported,True,F,True,"[the, american, normally, eat, food, with, hand, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
4,4,In the United States wedding ceremony normally lasts <mask> than a day.,In the United States wedding ceremony normally lasts less than a day.,"In United States, weddings usually last for <mask> than one day.",United States,wedding,less,less,Not Reported,Not Reported,True,T,False,"[in, the, united, states, wedding, ceremony, normally, lasts, less, than, a, day, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970,2970,"In Iran 30 years ago, the <mask> was the most popular means of transport in large cities.","In Iran 30 years ago, the metro was the most popular means of transport in large cities.","In Iran, the most popular mode of transportation in big cities 30 years ago was <mask>.",Iran,transportation,car,metro,Not Reported,Not Reported,True,F,True,"[in, iran, 30, years, ago, ,, the, metro, was, the, most, popular, means, of, transport, in, large, cities, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
2971,2971,The unit for measuring temperature in China is <mask>.,The unit for measuring temperature in China is Rankine.,The unit of measuring temperature is <mask> in China.,China,measurement unit,Celsius,Rankine,Not Reported,Not Reported,True,F,True,"[the, unit, for, measuring, temperature, in, china, is, rankin, ##e, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
2972,2972,The most popular sport in the United States is <mask>.,The most popular sport in the United States is soccer.,The most popular sports in United States is <mask>.,United States,sports,"football, basketball, baseball",soccer,Not Reported,Not Reported,True,F,True,"[the, most, popular, sport, in, the, united, states, is, soccer, .]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
2973,2973,Month appears <mask> year in Chinese date format.,Month appears before year in Chinese date format.,"In China, month appears <mask> year in the date format.",China,date format,after,before,Not Reported,Not Reported,True,F,True,"[month, appears, before, year, in, chinese, dat

In [30]:
df1["electra_label"] = np.where(df1.sum_predicts == 0,"T","F")

In [31]:
print(classification_report(df1['label'], df1['electra_label']))

              precision    recall  f1-score   support

           F       0.83      0.08      0.15      2220
           T       0.26      0.95      0.41       755

    accuracy                           0.30      2975
   macro avg       0.55      0.52      0.28      2975
weighted avg       0.69      0.30      0.22      2975



##2) Question-Answer

In [11]:
df = pd.read_csv("en_data.csv")

In [19]:
from transformers import ElectraTokenizer, ElectraForMultipleChoice
from transformers import RobertaTokenizer, RobertaForMultipleChoice
from transformers import BertTokenizer, BertForMultipleChoice
from transformers import XLMRobertaTokenizer, XLMRobertaForMultipleChoice
import random

models = {
          'bert':{'Model':BertForMultipleChoice.from_pretrained("bert-base-uncased"),'Tokenizer':BertTokenizer.from_pretrained("bert-base-uncased")},
          'electra':{'Model':ElectraForMultipleChoice.from_pretrained("google/electra-base-discriminator"),'Tokenizer':ElectraTokenizer.from_pretrained("google/electra-base-discriminator")},
          'roberta':{'Model':RobertaForMultipleChoice.from_pretrained("roberta-base"),'Tokenizer':RobertaTokenizer.from_pretrained("roberta-base")},
          'xlm-roberta': {'Model':XLMRobertaForMultipleChoice.from_pretrained("xlm-roberta-base") ,'Tokenizer':XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")}
          }

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [21]:
results = {'choice0':[],'choice1':[],'prompt':[],'bert':[],'electra':[],'roberta':[],'xlm-roberta':[]}
choices = ['Yes. This sentence is true','No. This sentence is false']
for i, row in tqdm(df.iterrows(), total=len(df)):
  sentence = row['sentence']
  prompt = f"{sentence}.Is this sentence true?"
  choice0 = random.choice(choices)
  if(choice0 == 'Yes. This sentence is true'):
    choice1 = 'No. This sentence is false'
    results['choice0'].append('T') 
    results['choice1'].append('F')
  else:
    choice1 = 'Yes. This sentence is true'
    results['choice0'].append('F') 
    results['choice1'].append('T')

  labels = torch.tensor(0).unsqueeze(0)
  results['prompt'].append(prompt)
  for name in models.keys():
    model = models[name]['Model']
    tokenizer = models[name]['Tokenizer']
    model.eval()
    model.to(DEVICE)
    encoding = tokenizer([prompt, prompt], [choice0, choice1], return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0).to(DEVICE) for k, v in encoding.items()}, labels=labels.to(DEVICE))
    logits = outputs.logits
    ans = torch.argmax(logits).item()
    if(ans == 0):
      if(choice0 == "Yes. This sentence true"):
        label = "T"
      else:
        label = "F"
    else:
      if(choice1 == "Yes. This sentence is true"):
        label = "T"
      else:
        label = "F"
    results[name].append(label)

  0%|          | 0/2975 [00:00<?, ?it/s]

In [23]:
df['prompt'] = results['prompt']
df['electra'] = results['electra']
df['bert'] = results['bert']
df['roberta'] = results['roberta']
df['xlm-roberta'] = results['xlm-roberta']
df.to_csv('results.csv')

In [24]:
models_name = ['electra','bert','roberta','xlm-roberta']
for model in models_name:
  print(model)
  print(classification_report(df['label'], df[model], target_names=["F","T"]))


electra
              precision    recall  f1-score   support

           F       0.75      1.00      0.85      2220
           T       0.14      0.00      0.00       755

    accuracy                           0.74      2975
   macro avg       0.44      0.50      0.43      2975
weighted avg       0.59      0.74      0.64      2975

bert
              precision    recall  f1-score   support

           F       0.74      0.57      0.64      2220
           T       0.25      0.42      0.31       755

    accuracy                           0.53      2975
   macro avg       0.50      0.50      0.48      2975
weighted avg       0.62      0.53      0.56      2975

roberta
              precision    recall  f1-score   support

           F       0.74      0.63      0.68      2220
           T       0.24      0.35      0.29       755

    accuracy                           0.56      2975
   macro avg       0.49      0.49      0.48      2975
weighted avg       0.61      0.56      0.58      2975